In [6]:
import pathlib
pathlib.Path().resolve()

PosixPath('/Users/garychen/Documents/Therapy-Chatbot-Deploying-NLP/Analyzing')

In [34]:
import jieba
import jieba.posseg as pseg
def jiebaSlice(content,mode):
    stopword_set = []
    with open('./stopword.txt','r', encoding='utf-8') as stopwords:
        for stopword in stopwords:
            stopword_set.append(stopword.strip('\n'))

    content = content.strip('\n')
    if mode == "POSSEG":
        words = pseg.cut(content,use_paddle=True)
        slicedWords = []
        for word, flag in words:
            if word not in stopword_set:
                slicedWords.append(word)
        return slicedWords
    elif mode == "CUT_HMM":
        seg_list = jieba.cut(content,HMM=True,cut_all=True)
        slicedWords = list(seg_list)
        return slicedWords
    elif mode == "CUT_FOR_SEARCH":
        seg_list = jieba.cut_for_search(content,HMM=True)
        slicedWords = list(seg_list)
        return slicedWords
    elif mode == "NORMAL":
        seg_list = jieba.cut_for_search(content)
        slicedWords = list(seg_list)
        return slicedWords

In [52]:
modes = ['POSSEG','CUT_HMM', 'CUT_FOR_SEARCH', 'NORMAL']
sentences = ['國家兩廳院發這樣的文，然後就不出意料的出事了']
for x in modes:
    for y in sentences:
        print(x,y)
        print(jiebaSlice(y,x))

POSSEG 國家兩廳院發這樣的文，然後就不出意料的出事了
['國家', '兩', '廳', '院', '發', '文', '後', '不出', '意料', '出事']
CUT_HMM 國家兩廳院發這樣的文，然後就不出意料的出事了
['國家', '兩廳', '院發', '這樣', '的', '文', '，', '然後就', '不出', '意料', '的', '出事', '了']
CUT_FOR_SEARCH 國家兩廳院發這樣的文，然後就不出意料的出事了
['國家', '兩廳', '院發', '這樣', '的', '文', '，', '然後就', '不出', '意料', '的', '出事', '了']
NORMAL 國家兩廳院發這樣的文，然後就不出意料的出事了
['國家', '兩廳', '院發', '這樣', '的', '文', '，', '然後就', '不出', '意料', '的', '出事', '了']


In [44]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese', do_lower_case=True)

In [50]:
sentences_tokens = list(map(lambda t: ['[CLS]'] + jiebaSlice(t,'NORMAL') + ['[SEP]'], sentences))
print(sentences_tokens)

[['[CLS]', '下雨', '雨天', '下雨天', '留客', '天留', '我', '不留', '[SEP]'], ['[CLS]', '我', '的', '研究', '生命', '還有', '多少', '年', '[SEP]'], ['[CLS]', '以上', '我們', '展示', '了', '如何', '應用', '中文', '斷詞', '到', '實作', '一個', '簡單', '的', '相似', '歌詞', '推薦', '系統', '[SEP]'], ['[CLS]', '結巴', '對', '於', '新', '詞辨識', '的', '表現', '還算', '不錯', '[SEP]']]


In [49]:
from keras_preprocessing.sequence import pad_sequences
sentences_tokens_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, sentences_tokens)), maxlen=512, truncating="post", padding="post", dtype="int")

In [51]:
print(sentences_tokens_ids)

[[ 101  100  100 ...    0    0    0]
 [ 101 2769 4638 ...    0    0    0]
 [ 101  100  100 ...    0    0    0]
 [ 101  100 2205 ...    0    0    0]]
